In [74]:
import requests
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

In [77]:
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.get("https://www.google.com")

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome
[WDM] - Driver [C:\Users\Patrick\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache


In [136]:
base_url = 'https://battlefieldtracker.com/bfv/leaderboards/stats/all/Wins?type=stats&page=1'

driver.get(base_url)

# Wait for page to load
time.sleep(15)

content = driver.page_source.encode('utf-8').strip()
soup = bs(content, 'html.parser')

The HTML here is actually pretty well organized.

Each player is a row in a table on the page. I want just the body, where the rows are, i.e. I can ignore the thead.

In [137]:
player_table = soup.find("table",{'class':'trn-table'}).tbody
player_rows = player_table.find_all('tr')

Each row in the table is itself a table. The rows of each player row proceed horizontally, left to right, as opposed to vertically. They are all named decently, too. There are a number of features to keep here: rank, username, stat highlight (the stat the players are organized by, here, wins) and stat-collapse (here, rounds played.) In addition, the player's platform can be retrieved from the relative path to their profile.

In [138]:
player_data = player_rows[-1].find_all('td')
player_data

[<td class="rank" data-v-4dbc6408="" data-v-754518f9=""><span data-v-4dbc6408="" data-v-754518f9="" data-v-7a5012a4="">100</span></td>,
 <td class="username" data-v-4dbc6408="" data-v-754518f9=""><div class="text" data-v-4dbc6408="" data-v-754518f9=""><a class="" data-v-4dbc6408="" data-v-754518f9="" href="/bfv/profile/psn/dark_throne957"><div class="avatar" data-v-4dbc6408=""><img alt="dark_throne957's Avatar" class="picture" data-v-4dbc6408="" loading="lazy" src="https://imgsvc.trackercdn.com/url/size(32)/https%3A%2F%2Fpsn-rsc.prod.dl.playstation.net%2Fpsn-rsc%2Favatar%2FUP4635%2FCUSA11631_00-AV00000000000002_E44F8FD9A023EE2386CA_l.png/image.jpg"/> <!-- --></div> <svg class="platform-icon platform-icon platform-psn" data-v-4dbc6408="" data-v-b17d31f6="" viewbox="5 117.952 990 764.096"><path d="M986.195 702.762c-19.338 24.398-66.718 41.803-66.718 41.803l-352.455 126.6V777.8l259.383-92.42c29.436-10.546 33.955-25.455 10.029-33.28-23.881-7.848-67.122-5.6-96.58 4.992l-172.832 60.871v-96.8

In [169]:
player_rows[0].find_all('td')[0].text

'\n  1\n'

In [178]:
bftracker_url = 'https://www.battlefieldtracker.com'
player_profile = bftracker_url + player_data[1].a['href']
player_profile

'https://www.battlefieldtracker.com/bfv/profile/psn/dark_throne957'

In [179]:
bftracker_url = 'https://www.battlefieldtracker.com'


def parse_player_row(player_data):
    player_rank = int(player_data[0].text.strip())
    player_username = player_data[1].find('span', {'class':'trn-ign__username'}).text.strip()
    player_platform = player_data[1].a['href'].split('/')[-2]
    player_wins = int(player_data[-2].text.strip().replace(',', ''))
    player_rounds_played =  int(player_data[-1].text.strip().replace(',', ''))
    player_profile = bftracker_url + player_data[1].a['href']
    player_features = [player_rank, player_username, player_platform, player_wins, player_rounds_played, player_profile]
    return player_features

features = []
for i,row in enumerate(player_rows):
    player_data = row.find_all('td')
    features.append(parse_player_row(player_data))
    
print('First 3 rows')
print(features[:3])

print('Last 3 rows')
features[-3:]

First 3 rows
[[1, 'themadbat2', 'psn', 20519, 27856, 'https://www.battlefieldtracker.com/bfv/profile/psn/themadbat2'], [2, 'lHluslHlKuslHl', 'psn', 17698, 19754, 'https://www.battlefieldtracker.com/bfv/profile/psn/lHluslHlKuslHl'], [3, 'jawasandcrawler', 'psn', 17477, 29835, 'https://www.battlefieldtracker.com/bfv/profile/psn/jawasandcrawler']]
Last 3 rows


[[98,
  'WebWire',
  'psn',
  9959,
  16503,
  'https://www.battlefieldtracker.com/bfv/profile/psn/WebWire'],
 [99,
  'TightWolf',
  'xbl',
  9953,
  19693,
  'https://www.battlefieldtracker.com/bfv/profile/xbl/TightWolf'],
 [100,
  'dark_throne957',
  'psn',
  9923,
  15282,
  'https://www.battlefieldtracker.com/bfv/profile/psn/dark_throne957']]

In [189]:
profiles = np.asarray(features)[:,5]
req = requests.get('https://battlefieldtracker.com/bfv/profile/psn/themadbat2/overview').content
soup2 = bs(req)

In [192]:
driver.get(profiles[0])

# Wait for page to load
time.sleep(5)

content = driver.page_source.encode('utf-8').strip()
profile_soup = bs(content, 'html.parser')

There are a lot of player stats on each profile page, both overall and broken down by class. The easiest way toget these is using css selectors, for the most part.

In [193]:
profile_soup

<html class="" data-vm-ssr="" lang="en"><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, user-scalable=no, viewport-fit=cover" name="viewport"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<title>themadbat2's Battlefield V Overview Stats - Battlefield Tracker</title>
<meta content="app-id=network.tracker.app" name="google-play-app"/>
<meta content="app-id=1287696482" name="apple-itunes-app"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="yes" name="mobile-web-app-capable"/>
<meta content="black-translucent" name="apple-mobile-web-app-status-bar-style"/>
<meta content="/public/browserconfig.xml" name="msapplication-config"/>
<meta content="#101921" name="msapplication-TileColor"/>
<meta content="/public/icons/tile310.png" name="msapplication-TileImage"/>
<meta content="Battlefield Tracker" data-vm="1" name="application-name"/><meta content="#1b2640" data-vm="1" name="theme-color"/><meta content="Battlefield Tracker"